In [ ]:
import pandas as pd
from ensemble_boxes import *

In [ ]:
!mkdir results

In [ ]:
# csv 파일 로드
file1 = './mmdetection/work_dirs/01_swin_bbox_safe/submission_best_bbox_mAP_epoch_16.csv'
file2 = './mmdetection/work_dirs/cascade-mask-rcnn_convnext-t/submission_best_bbox_mAP_epoch_20.csv'
file3 = './mmdetection/work_dirs/02_res2net101_bbox_safe/submission_best_bbox_mAP_epoch_16.csv'
output_file = './results/ensemble.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

df1['source'] = file1
df2['source'] = file2
df3['source'] = file3


the_weight = [0.4, 0.3, 0.3]    # wbf ensemble 비율 설정


# 세 DataFrame을 하나로 결합
df = pd.concat([df1, df2, df3])

# 이미지의 너비와 높이
IMAGE_WIDTH = 1920
IMAGE_HEIGHT = 1080

def convert_boxes(row):
    return [(row['point1_x'] / IMAGE_WIDTH), (row['point1_y'] / IMAGE_HEIGHT), 
            (row['point3_x'] / IMAGE_WIDTH), (row['point3_y'] / IMAGE_HEIGHT)]


df['boxes'] = df.apply(lambda row: convert_boxes(row), axis=1)

In [ ]:
# 각 이미지에 대한 앙상블
ensemble_results = []

grouped = df.groupby('file_name')

for name, group in grouped:
    scores_list = []
    boxes_list = []
    labels_list = []
    weights = []
    
    g2 = group.groupby('source')
    for source, group2 in g2:
        if source == file1:
            idx = 0
        elif source == file2:
            idx = 1
        else:
            idx = 2
        weights.append(the_weight[idx])

        score_t = group2['confidence'].values.tolist()
        box_t = group2['boxes'].values.tolist()
        label_t = group2['class_id'].values.tolist()

        scores_list.append(score_t)
        boxes_list.append(box_t)
        labels_list.append(label_t)
    
    

    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights = weights, iou_thr=0.5, skip_box_thr=0.049)
    for box, score, label in zip(boxes, scores, labels):
        ensemble_results.append({
            'file_name': name,
            'class_id': int(label),
            'confidence': score,
            'point1_x': box[0] * IMAGE_WIDTH,
            'point1_y': box[1] * IMAGE_HEIGHT,
            'point2_x': box[2] * IMAGE_WIDTH,
            'point2_y': box[1] * IMAGE_HEIGHT,
            'point3_x': box[2] * IMAGE_WIDTH,
            'point3_y': box[3] * IMAGE_HEIGHT,
            'point4_x': box[0] * IMAGE_WIDTH,
            'point4_y': box[3] * IMAGE_HEIGHT,
        })

ensemble_df = pd.DataFrame(ensemble_results)

In [ ]:
# 앙상블 결과를 csv 파일로 저장
ensemble_df.to_csv(output_file, index=False)
print(len(ensemble_df))